In [11]:
import json
from datetime import datetime
from collections import defaultdict

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import helper

%matplotlib inline
plt.rcdefaults()
plt.rcParams.update({'font.size': 14})
plt.style.use('seaborn-whitegrid')

In [2]:
# Request all resources available on Transifex
resources = helper.transifex_api("resources/")
resources = [resource["slug"] for resource in resources]

In [33]:
# Retrieve statistics from every resource
# TODO: We'd benefit from asyncio here. It's a bit slow and
# be able to run concurrent request would help make it faster.

fields = [
    "reviewed",
    "translated_entities",
    "translated_words",
    "untranslated_entities",
    "untranslated_words",
]
stats = defaultdict(lambda: {field: 0 for field in fields})
for resource in resources:
    stat = helper.transifex_api(f"resource/{resource}/stats/pt_BR/")
    
    # Group stats considering the first part of slug.
    # Transform "c-api--abstract" and "c-api--allocation" into "c-api".
    resource = resource.split('--')[0]
    
    for field in fields:
        stats[resource][field] += stat[field]

In [39]:
stats["glossary"] = stats["glossary_"]
del stats["glossary_"]
stats = {key: stats[key] for key in sorted(stats.keys(), reverse=True)}

In [34]:
today = datetime.now().strftime("%Y_%m_%d")
filename = f"data/{today}.json"

In [35]:
with open(filename, mode="w") as stat_file:
    stat_file.write(json.dumps(stats))